### Importando bibliotecas

In [15]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%pip install pgmpy
import pgmpy
from datetime import date

Note: you may need to restart the kernel to use updated packages.


### Definindo o Dataframe

In [16]:
df = pd.read_csv('republican_democrat.csv', sep = ',')
print(df)

    handicapped-infants water-project-cost-sharing  \
0                     n                          y   
1                     n                          y   
2                     ?                          y   
3                     n                          y   
4                     y                          y   
..                  ...                        ...   
410                   ?                          ?   
411                   y                          n   
412                   y                          y   
413                   y                          y   
414                   n                          y   

    adoption-of-the-budget-resolution physician-fee-freeze  \
0                                   n                    y   
1                                   n                    y   
2                                   y                    ?   
3                                   y                    n   
4                                   y    

### Avaliando a quantidade de valores faltantes de cada atributo


In [17]:
print(df.value_counts())

handicapped-infants  water-project-cost-sharing  adoption-of-the-budget-resolution  physician-fee-freeze  religious-groups-in-schools  anti-satellite-test-ban  mx-missile  immigration  synfuels-corporation-cutback  education-spending  superfund-right-to-sue  crime  duty-free-exports  export-administration-act-south-africa  Target    
n                    y                           n                                  y                     y                            n                        n           n            n                             y                   y                       y      n                  y                                       republican    9
                     n                           n                                  y                     y                            n                        n           n            n                             y                   y                       y      n                  n                                     



Vemos que, em alguns atributos, está listado '?' como valor, o que indica que há dados faltantes. Como representam uma pequena porcentagem, é possível excluir as linhas com valores faltantes sem grande perda de dados.

In [18]:
for col in list(df):        # itera sobre as colunas de df
    df=df[df[col]!='?']     # para cada coluna, elimina linhas  com valor '?'
print(df)

    handicapped-infants water-project-cost-sharing  \
3                     n                          y   
5                     n                          y   
8                     n                          y   
19                    y                          y   
23                    y                          y   
..                  ...                        ...   
406                   y                          n   
407                   n                          n   
409                   y                          n   
412                   y                          y   
413                   y                          y   

    adoption-of-the-budget-resolution physician-fee-freeze  \
3                                   y                    n   
5                                   y                    n   
8                                   n                    y   
19                                  y                    n   
23                                  y    


### Construção da Rede Bayesiana

In [20]:
from pgmpy.factors.discrete import TabularCPD

# cpd - conditional probability distribuition
target_cpd = TabularCPD(variable='Target', variable_card=2, values=[[0.5], [0.5]])
print(target_cpd)

+-----------+-----+
| Target(0) | 0.5 |
+-----------+-----+
| Target(1) | 0.5 |
+-----------+-----+
